In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from tensorflow.keras.losses import binary_crossentropy

2021-11-14 21:39:07.842452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:39:07.842495: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def save(name, words, vectors, path='.'):
    with open(f'{path}/{name}.txt', 'w+', encoding='utf-8') as doc:
        for word, vector in zip(words, vectors):
            doc.write(word + ' ' + ' '.join(str(value) for value in vector))
            doc.write('\n')

In [3]:
import os
import pickle

def load_embeddings(file_name, vocabulary):
    """
    Loads word embeddings from the file with the given name.
    :param file_name: name of the file containing word embeddings
    :type file_name: str
    :param vocabulary: captions vocabulary
    :type vocabulary: numpy.array
    :return: word embeddings
    :rtype: dict
    """
    embeddings = dict()
    with open(file_name, 'r', encoding='utf-8') as doc:
        line = doc.readline()
        while line != '':
            line = line.rstrip('\n').lower()
            parts = line.split(' ')
            vals = np.array(parts[1:], dtype=np.float)
            if parts[0] in vocabulary:
                embeddings[parts[0]] = vals
            line = doc.readline()
    return embeddings


def load_embedding_weights(vocabulary, embedding_size, embedding_type, path='.'):
    print("local")
    """
    Creates and loads embedding weights.
    :param vocabulary: vocabulary
    :type vocabulary: numpy.array
    :param embedding_size: embedding size
    :type embedding_size: int
    :param embedding_type: type of the pre-trained embeddings
    :type embedding_type: string
    :return: embedding weights
    :rtype: numpy.array
    """
    if os.path.exists(f'{path}/embedding_matrix_{embedding_type}_{embedding_size}.pkl'):
        with open(f'{path}/embedding_matrix_{embedding_type}_{embedding_size}.pkl', 'rb') as f:
            embedding_matrix = pickle.load(f)
    else:
        print('Creating embedding weights...')
        if embedding_type == 'glove':
            embeddings = load_embeddings(f'{path}/glove.6B.{embedding_size}d.txt', vocabulary)
        else:
          embeddings = load_embeddings(f'{path}/word2vecSG.iSarcasamEval.{embedding_size}d.txt', vocabulary)
        embedding_matrix = np.zeros((len(vocabulary), embedding_size))
        for i in range(len(vocabulary)):
            if vocabulary[i] in embeddings.keys():
                embedding_matrix[i] = embeddings[vocabulary[i]]
            else:
                embedding_matrix[i] = np.random.standard_normal(embedding_size)
        with open(f'{path}/embedding_matrix_{embedding_type}_{embedding_size}.pkl', 'wb') as f:
            pickle.dump(embedding_matrix, f)
    return embedding_matrix

from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
def create_vocabulary(sentence_tokens):
    vocabulary = set()
    for tokens in sentence_tokens:
        vocabulary.update(tokens)

    vocabulary = list(vocabulary)
    word_to_id = {word: index for word, index in zip(vocabulary, range(len(vocabulary)))}
    return vocabulary, word_to_id

# Load Dataset

In [5]:
df = pd.read_csv('data/trial.csv',delimiter="	")
df.columns = ['file_name', 'misogynous', 'shaming', 'stereotype', 'objectification', 'violence', 'text_transcription']
df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,text_transcription
0,28.jpg,0,0,0,0,0,"not now, dad. We should burn Jon Snow. stop it..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat
3,58.jpg,1,0,0,0,1,"Bitches be like, It was my fault i made him mad"
4,89.jpg,0,0,0,0,0,find a picture of 4 girls together on FB make ...


# Create Embeddings

In [6]:
from nltk import word_tokenize
from nltk.corpus import stopwords
ENGLISH_STOPWORDS = stopwords.words('english')

def remove_punctuation(token:str)->str:
    punctuation_regex = '!"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    return ' '.join(word.strip(punctuation_regex) for word in token.split())

def nlp_pipeline(token:str) -> str:
    token = remove_punctuation(token)
    tokens = word_tokenize(token.lower())
    tokens = [token for token in tokens if token not in ENGLISH_STOPWORDS]
    return tokens

df['text_transcription_tokens'] = df['text_transcription'].apply(lambda x: nlp_pipeline(x))
df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,text_transcription,text_transcription_tokens
0,28.jpg,0,0,0,0,0,"not now, dad. We should burn Jon Snow. stop it...","[dad, burn, jon, snow, stop, dad, know, happen..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...,"[may, mixcommunication, decorator, happy, birt..."
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat,"[n't, sold, boat]"
3,58.jpg,1,0,0,0,1,"Bitches be like, It was my fault i made him mad","[bitches, like, fault, made, mad]"
4,89.jpg,0,0,0,0,0,find a picture of 4 girls together on FB make ...,"[find, picture, 4, girls, together, fb, make, ..."


## Word2Vec

In [7]:
sentences = df['text_transcription_tokens'].values

def build_embeddings(model_name, sentences, size, path):
    if model_name == 'word2vec':
        model = Word2Vec(sentences, vector_size=size, min_count=1, window=5, sg=1)
        vectors = model.wv.vectors      
        words = model.wv.index_to_key

        save(f'{model_name}SG.iSarcasamEval.{size}d', words, vectors,path=path)

for size in [10, 50, 100]:
    build_embeddings('word2vec', sentences, size=size, path='.')

# Task 1

## Tokenization and vocabulary creation

In [8]:
sentences = df['text_transcription_tokens'].values

vocabulary, word_to_id = create_vocabulary(sentences)
vocabulary_size = len(vocabulary)
max_length = max(map(lambda x: len(x), word_to_id.keys()))
X = df['text_transcription_tokens'].apply(lambda x: np.array([word_to_id[i] for i in x])).values

X_pad = pad_sequences(X, maxlen=max_length, padding='post')
y = df['misogynous']

## Pre-trained Weights

In [9]:
embeddings = {
    50: load_embedding_weights(vocabulary, 50, 'word2vecSG', ".")}

local


In [10]:
glove_50 = load_embedding_weights(vocabulary, 50, 'glove', "/mnt/d/Downloads")

local


## Define Models

In [11]:
import wandb
from wandb.keras import WandbCallback

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.1)

### Without initialized weights

In [13]:
model = Sequential(name="without_initialized_weights.task1")
model.add(Embedding(input_dim = vocabulary_size, output_dim=100))
model.add(LSTM(units=100, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate = 0.01), loss=binary_crossentropy, metrics=['accuracy',f1_m,precision_m, recall_m])
model.summary()

2021-11-14 21:39:31.099441: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-14 21:39:31.099482: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-14 21:39:31.099525: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (legion-y540): /proc/driver/nvidia/version does not exist
2021-11-14 21:39:31.099834: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "without_initialized_weights.task1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         47000     
                                                                 
 lstm (LSTM)                 (None, None, 100)         80400     
                                                                 
 dense (Dense)               (None, None, 1)           101       
                                                                 
Total params: 127,501
Trainable params: 127,501
Non-trainable params: 0
_________________________________________________________________


In [14]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=100, callbacks=[WandbCallback()])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aleksandar1932 (use `wandb login --relogin` to force relogin)
/home/aleksandar/envs/nlp-2021-n/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
2021-11-14 21:39:36.637687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:39:36.637762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/100
3/3 [==============================] - 2s 57ms/step - loss: 0.7015 - accuracy: 0.5306 - f1_m: 0.5752 - precision_m: 0.3789 - recall_m: 2.1349
Epoch 2/100
3/3 [==============================] - 0s 34ms/step - loss: 0.6911 - accuracy: 0.5523 - f1_m: 1.2321 - precision_m: 0.6472 - recall_m: 24.3691
Epoch 3/100
3/3 [==============================] - 0s 34ms/step - loss: 0.6420 - accuracy: 0.7121 - f1_m: 1.8009 - precision_m: 0.9679 - recall_m: 13.4485
Epoch 4/100
3/3 [==============================] - 0s 34ms/step - loss: 0.1950 - accuracy: 0.9831 - f1_m: 1.8869 - precision_m: 0.9713 - recall_m: 32.9257
Epoch 5/100
3/3 [==============================] - 0s 33ms/step - loss: 0.0376 - accuracy: 0.9948 - f1_m: 1.9201 - precision_m: 0.9889 - recall_m: 32.9722
Epoch 6/100
3/3 [==============================] - 0s 30ms/step - loss: 0.0180 - accuracy: 0.9966 - f1_m: 1.9289 - precision_m: 0.9935 - recall_m: 32.9778
Epoch 7/100
3/3 [==============================] - 0s 29ms/step - loss:

In [15]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test f1_score: {f1_score:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')

run.finish()

Test loss: 6.9088
Test accuracy: 0.5030
Test f1_score: 0.6600
Test precision: 0.3350
Test recall: 22.0000


accuracy,▁▄██████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_m,▁▇██████████████████████████████████████
loss,█▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision_m,▁███████████████████████████████████████
recall_m,▁▄██████████████████████████████████████
accuracy,0.99862
epoch,99
f1_m,1.93687
loss,0.00219
precision_m,0.99773


### Word2Vec 50

In [16]:
model = Sequential(name="word2vec.50d.task1")

model.add(Embedding(input_dim = vocabulary_size, weights=[embeddings[50]], output_dim=50))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.01), loss=binary_crossentropy, metrics=['accuracy', f1_m,precision_m, recall_m])
model.summary()

Model: "word2vec.50d.task1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          23500     
                                                                 
 lstm_1 (LSTM)               (None, None, 64)          29440     
                                                                 
 lstm_2 (LSTM)               (None, None, 128)         98816     
                                                                 
 dense_1 (Dense)             (None, None, 1)           129       
                                                                 
Total params: 151,885
Trainable params: 151,885
Non-trainable params: 0
_________________________________________________________________


In [17]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=50, callbacks=[WandbCallback()])

2021-11-14 21:40:39.145566: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:40:39.145613: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/50
3/3 [==============================] - 3s 83ms/step - loss: 0.7005 - accuracy: 0.5010 - f1_m: 0.5923 - precision_m: 0.3034 - recall_m: 13.7468
Epoch 2/50
3/3 [==============================] - 0s 59ms/step - loss: 0.6955 - accuracy: 0.5145 - f1_m: 0.2767 - precision_m: 0.1429 - recall_m: 4.3846
Epoch 3/50
3/3 [==============================] - 0s 125ms/step - loss: 0.6757 - accuracy: 0.5937 - f1_m: 0.6268 - precision_m: 0.3333 - recall_m: 5.2424
Epoch 4/50
3/3 [==============================] - 0s 77ms/step - loss: 0.3176 - accuracy: 0.9322 - f1_m: 1.8300 - precision_m: 0.9437 - recall_m: 30.1557
Epoch 5/50
3/3 [==============================] - 0s 61ms/step - loss: 0.0984 - accuracy: 0.9769 - f1_m: 1.9083 - precision_m: 0.9839 - recall_m: 31.6174
Epoch 6/50
3/3 [==============================] - 0s 56ms/step - loss: 0.0329 - accuracy: 0.9962 - f1_m: 1.9365 - precision_m: 0.9977 - recall_m: 32.8125
Epoch 7/50
3/3 [==============================] - 0s 59ms/step - loss: 0.0198

In [18]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test f1_score: {f1_score:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')

run.finish()

Test loss: 6.2160
Test accuracy: 0.5697
Test f1_score: 0.7051
Test precision: 0.3595
Test recall: 18.3333


accuracy,▁▁▂▇████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
f1_m,▂▁▂█████████████████████████████████████
loss,███▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision_m,▂▁▃█████████████████████████████████████
recall_m,▃▁▁▇████████████████████████████████████
accuracy,0.99862
epoch,49
f1_m,1.93869
loss,0.00237
precision_m,0.99873


### GloVe 50

In [19]:
model = Sequential(name="glove.50d.task1")

model.add(Embedding(input_dim = vocabulary_size, output_dim=50))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.05), loss=binary_crossentropy, metrics=['accuracy',f1_m,precision_m, recall_m])
model.summary()

Model: "glove.50d.task1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          23500     
                                                                 
 lstm_3 (LSTM)               (None, None, 128)         91648     
                                                                 
 dense_2 (Dense)             (None, None, 1)           129       
                                                                 
Total params: 115,277
Trainable params: 115,277
Non-trainable params: 0
_________________________________________________________________


In [20]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=50, callbacks=[WandbCallback()])

2021-11-14 21:41:16.595611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:41:16.595653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/50
3/3 [==============================] - 2s 48ms/step - loss: 0.8331 - accuracy: 0.5165 - f1_m: 0.6226 - precision_m: 0.3173 - recall_m: 16.5152
Epoch 2/50
3/3 [==============================] - 0s 36ms/step - loss: 0.8549 - accuracy: 0.4742 - f1_m: 0.7736 - precision_m: 0.6357 - recall_m: 10.2955
Epoch 3/50
3/3 [==============================] - 0s 34ms/step - loss: 0.7788 - accuracy: 0.4666 - f1_m: 0.8822 - precision_m: 0.4479 - recall_m: 32.0000
Epoch 4/50
3/3 [==============================] - 0s 51ms/step - loss: 0.7365 - accuracy: 0.5372 - f1_m: 0.6617 - precision_m: 0.5347 - recall_m: 0.8693
Epoch 5/50
3/3 [==============================] - 0s 33ms/step - loss: 0.7389 - accuracy: 0.5155 - f1_m: 0.6907 - precision_m: 0.4838 - recall_m: 11.3758
Epoch 6/50
3/3 [==============================] - 0s 37ms/step - loss: 0.7595 - accuracy: 0.4762 - f1_m: 0.9131 - precision_m: 0.4632 - recall_m: 32.9206
Epoch 7/50
3/3 [==============================] - 0s 39ms/step - loss: 0.8280

In [22]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test f1_score: {f1_score:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')

Test loss: 3.0158
Test accuracy: 0.4727
Test f1_score: 0.7174
Test precision: 0.3626
Test recall: 33.0000


In [23]:
run.finish()

accuracy,▂▁▁▂▁▂▁▁▂▂▄▃▇▆▅▇▆███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
f1_m,▁▂▂▁▃▁▂▃▃▃▅▅▇█▇▇▇▇██████████████████████
loss,██▇▇▇█▇▇▆▆▆▆▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision_m,▁▄▂▃▂▅▄▂▆▃▅▇▇█▇▇▇▇██████████████████████
recall_m,▄▃█▁█▁▃█▁█▆▃█▅▅█▆███████████████████████
accuracy,0.99862
epoch,49
f1_m,1.94001
loss,0.00415
precision_m,0.99944


## Evaluate Models

WANDB was used to track the training process along with the evaluation, all of the results are available at the [link](https://wandb.ai/aleksandar1932/NLP_2021-Laboratory%20Exercises_2?workspace=user-aleksandar1932).

*Runs are postfixed with `.task1`*.

# Task 2

## Tokenization and vocabulary creation

In [24]:
df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,text_transcription,text_transcription_tokens
0,28.jpg,0,0,0,0,0,"not now, dad. We should burn Jon Snow. stop it...","[dad, burn, jon, snow, stop, dad, know, happen..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...,"[may, mixcommunication, decorator, happy, birt..."
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat,"[n't, sold, boat]"
3,58.jpg,1,0,0,0,1,"Bitches be like, It was my fault i made him mad","[bitches, like, fault, made, mad]"
4,89.jpg,0,0,0,0,0,find a picture of 4 girls together on FB make ...,"[find, picture, 4, girls, together, fb, make, ..."


In [62]:
sentences = df['text_transcription_tokens'].values

In [25]:
vocabulary, word_to_id = create_vocabulary(sentences)
vocabulary_size = len(vocabulary)
max_length = max(map(lambda x: len(x), word_to_id.keys()))
X = df['text_transcription_tokens'].apply(lambda x: np.array([word_to_id[i] for i in x])).values
X_pad = pad_sequences(X, maxlen=max_length, padding='post')

In [26]:
y = df.iloc[:,2:6].values.astype('float').reshape((-1,4))

## Pre-trained Weights

## Define Models

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.1)

### Without Pre-Trained Weights

In [28]:
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy

In [41]:
model = Sequential(name="withoutInitializedWeights.misogenyType")
model.add(Embedding(input_dim=vocabulary_size, output_dim=50))
model.add(LSTM(units=128))
model.add(Dense(4,activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.05), loss=categorical_crossentropy,  metrics=['accuracy',f1_m,precision_m, recall_m])
model.summary()
model.output_shape

Model: "withoutInitializedWeights.misogenyType"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 50)          23500     
                                                                 
 lstm_11 (LSTM)              (None, 128)               91648     
                                                                 
 dense_10 (Dense)            (None, 4)                 516       
                                                                 
Total params: 115,664
Trainable params: 115,664
Non-trainable params: 0
_________________________________________________________________


(None, 4)

In [42]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=200, batch_size=10, callbacks=[WandbCallback()])

2021-11-14 21:50:00.262228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:50:00.262265: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/200
9/9 [==============================] - 2s 35ms/step - loss: 0.4436 - accuracy: 0.2159 - f1_m: 0.2692 - precision_m: 0.2000 - recall_m: 0.4360
Epoch 2/200
9/9 [==============================] - 0s 27ms/step - loss: 0.4799 - accuracy: 0.2045 - f1_m: 0.2000 - precision_m: 0.1444 - recall_m: 0.3556
Epoch 3/200
9/9 [==============================] - 0s 26ms/step - loss: 0.3673 - accuracy: 0.2841 - f1_m: 0.3439 - precision_m: 0.2556 - recall_m: 0.6506
Epoch 4/200
9/9 [==============================] - 0s 26ms/step - loss: 0.3661 - accuracy: 0.3295 - f1_m: 0.3298 - precision_m: 0.2444 - recall_m: 0.5167
Epoch 5/200
9/9 [==============================] - 0s 26ms/step - loss: 0.3562 - accuracy: 0.3295 - f1_m: 0.2978 - precision_m: 0.2194 - recall_m: 0.4778
Epoch 6/200
9/9 [==============================] - 0s 27ms/step - loss: 0.3494 - accuracy: 0.3068 - f1_m: 0.2741 - precision_m: 0.2028 - recall_m: 0.4537
Epoch 7/200
9/9 [==============================] - 0s 30ms/step - loss: 0.36

In [43]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test f1_score: {f1_score:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')

run.finish()

Test loss: 0.3715
Test accuracy: 0.5000
Test f1_score: 0.6250
Test precision: 0.5000
Test recall: 0.8333


accuracy,▁▇████▇█▅█▄▅▅█▃█▄██▅▅▂▇▄▇████▂▅████▇▄██▄
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_m,▁▃▆▆▇▅▂▄▃█▃▄▃█▄█▃▃▄▅▂▃▄▅▇▆▆▆▅▄▇█▆▆▆▃▃▅▅▂
loss,█▁▂▂▁▂▄▂▂▂▅▄▂▂▄▁▃▂▂▃▂▃▂▆▂▂▂▂▂▅▁▂▄▁▂▂▂▂▁▃
precision_m,▁▃▆▆▇▅▂▅▄█▃▅▃█▄█▃▂▄▆▃▃▄▅▇▆▆▆▅▄▇█▆▇▆▄▃▅▅▂
recall_m,▁▃▆▆▆▄▂▄▃▇▃▃▂█▄█▃▄▃▅▁▃▃▄▇▅▆▆▅▃▇▇▅▆▆▃▃▆▄▁
accuracy,0.26136
epoch,199
f1_m,0.22568
loss,0.38463
precision_m,0.16944


### Word2Vec 50

In [44]:
model = Sequential(name="word2vec.50d.misogenyType")
model.add(Embedding(input_dim=len(vocabulary), output_dim=50, weights=[embeddings[50]]))
model.add(LSTM(units=64, activation='relu'))
model.add(Dense(4,activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.05), loss=categorical_crossentropy,  metrics=['accuracy',f1_m,precision_m, recall_m])
model.summary()
model.output_shape

Model: "word2vec.50d.misogenyType"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 50)          23500     
                                                                 
 lstm_12 (LSTM)              (None, 64)                29440     
                                                                 
 dense_11 (Dense)            (None, 4)                 260       
                                                                 
Total params: 53,200
Trainable params: 53,200
Non-trainable params: 0
_________________________________________________________________


(None, 4)

In [45]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=200, callbacks=[WandbCallback()])

2021-11-14 21:51:15.527647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:51:15.527690: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/200
3/3 [==============================] - 1s 42ms/step - loss: 3.4588 - accuracy: 0.3295 - f1_m: 0.1333 - precision_m: 0.0938 - recall_m: 0.2308
Epoch 2/200
3/3 [==============================] - 0s 31ms/step - loss: 51.8492 - accuracy: 0.2045 - f1_m: 0.0145 - precision_m: 0.0104 - recall_m: 0.0238
Epoch 3/200
3/3 [==============================] - 0s 32ms/step - loss: 0.5183 - accuracy: 0.3295 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 4/200
3/3 [==============================] - 0s 30ms/step - loss: 332.3225 - accuracy: 0.1591 - f1_m: 0.2188 - precision_m: 0.1562 - recall_m: 0.3661
Epoch 5/200
3/3 [==============================] - 0s 29ms/step - loss: 19613944.0000 - accuracy: 0.2045 - f1_m: 0.2546 - precision_m: 0.1979 - recall_m: 0.3704
Epoch 6/200
3/3 [==============================] - 0s 30ms/step - loss: 15316484737728512.0000 - accuracy: 0.0909 - f1_m: 0.1206 - precision_m: 0.0868 - recall_m: 0.2020
Epoch 7/200
3/3 [=====================

In [46]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test f1_score: {f1_score:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')

run.finish()

Test loss: nan
Test accuracy: 0.5000
Test f1_score: nan
Test precision: nan
Test recall: nan


accuracy,▃▁██████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_m,▁█
loss,▁█
precision_m,▁█
recall_m,▁█
accuracy,0.59091
epoch,199
f1_m,nan
loss,nan
precision_m,nan


### GloVe 50

In [47]:
model = Sequential(name="word2vec.50d.misogenyType")
model.add(Embedding(input_dim=len(vocabulary), output_dim=50, weights=[embeddings[50]]))
model.add(LSTM(units=64, activation='relu'))
model.add(Dense(4,activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.05), loss=categorical_crossentropy,  metrics=['accuracy',f1_m,precision_m, recall_m])
model.summary()
model.output_shape

Model: "word2vec.50d.misogenyType"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 50)          23500     
                                                                 
 lstm_13 (LSTM)              (None, 64)                29440     
                                                                 
 dense_12 (Dense)            (None, 4)                 260       
                                                                 
Total params: 53,200
Trainable params: 53,200
Non-trainable params: 0
_________________________________________________________________


(None, 4)

In [48]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=200, callbacks=[WandbCallback()])

2021-11-14 21:51:50.906277: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-14 21:51:50.906329: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/200
3/3 [==============================] - 1s 40ms/step - loss: 1.1923 - accuracy: 0.1136 - f1_m: 0.1554 - precision_m: 0.1076 - recall_m: 0.2803
Epoch 2/200
3/3 [==============================] - 0s 27ms/step - loss: 0.5316 - accuracy: 0.3295 - f1_m: 0.1250 - precision_m: 0.0870 - recall_m: 0.2222
Epoch 3/200
3/3 [==============================] - 0s 25ms/step - loss: 5.5144 - accuracy: 0.3295 - f1_m: 0.3422 - precision_m: 0.2465 - recall_m: 0.5671       
Epoch 4/200
3/3 [==============================] - 0s 27ms/step - loss: 23.9572 - accuracy: 0.3182 - f1_m: 0.3311 - precision_m: 0.2788 - recall_m: 0.4263
Epoch 5/200
3/3 [==============================] - 0s 24ms/step - loss: 3752335872.0000 - accuracy: 0.3068 - f1_m: 0.3934 - precision_m: 0.3223 - recall_m: 0.5358
Epoch 6/200
3/3 [==============================] - 0s 25ms/step - loss: 30666157752905105408.0000 - accuracy: 0.3295 - f1_m: 0.4314 - precision_m: 0.3364 - recall_m: 0.6098
Epoch 7/200
3/3 [=======================

In [49]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test f1_score: {f1_score:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')

run.finish()

Test loss: nan
Test accuracy: 0.5000
Test f1_score: nan
Test precision: nan
Test recall: nan


accuracy,▁▁██████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_m,▁█
loss,▁█
precision_m,▁█
recall_m,▁█
accuracy,0.59091
epoch,199
f1_m,nan
loss,nan
precision_m,nan


## Evaluate Models

WANDB was used to track the training process along with the evaluation, all of the results are available at the [link](https://wandb.ai/aleksandar1932/NLP_2021-Laboratory%20Exercises_2?workspace=user-aleksandar1932).

*Runs are postfixed with `.misogenyType`*.